## Análisis de accidentes - Buses

### Area metropolitana del Valle de Aburrá

Manuela Cordoba

Alejandro Fernandez

Ana María Jaramillo


In [1]:
import pandas as pd
import config

In [2]:
def get_categories_with_number(row: pd.Series) -> str:
    """
    This function returns a string with the categories that have a value different from 0
    Args:
        row (pd.Series): row of the dataframe
    Returns:
        str: string with the categories that have a value different from 0
    """
    categories = []
    for col, value in row.items():
        if col != "radicado" and value != 0:
            categories.append(col)
    return "-".join(categories) if categories else ""

In [3]:
cleaned_data = pd.read_excel(config.CLEANED_DATA)
cleaned_data.head()

,radicado,periodo,tipo_geoco,gravedad,barrio,mes,dia,longitud,latitud,areagralus,...,clase_vehiculo,tipo_servicio,marca,modelo,zona,diseno,edad_conductor,sexo,dia_semana,hora
0,1739400,2021,malla_vial,herido,prado,2,25,-75.563376,6.263505,areas_y_corredores_de_alta_mixtura,...,bus,publico,modasa,2011.0,comuna_3,glorieta,45.0,masculino,jue,14
1,1735512,2021,malla_vial,solo_danos,el_progreso,1,14,-75.578933,6.271674,areas_y_corredores_de_media_mixtura,...,automovil,particular,mazda,2011.0,comuna_7,tramo_de_via,30.0,masculino,jue,18
2,1735512,2021,malla_vial,solo_danos,el_progreso,1,14,-75.578933,6.271674,areas_y_corredores_de_media_mixtura,...,bus,publico,agrale,2008.0,comuna_7,tramo_de_via,58.0,masculino,jue,18
3,1735374,2021,malla_vial,solo_danos,caribe,1,13,-75.575081,6.266311,areas_y_corredores_de_alta_mixtura,...,automovil,particular,renault,2017.0,comuna_7,tramo_de_via,55.0,masculino,mie,19
4,1735374,2021,malla_vial,solo_danos,caribe,1,13,-75.575081,6.266311,areas_y_corredores_de_alta_mixtura,...,bus,publico,chevrolet,2015.0,comuna_7,tramo_de_via,30.0,masculino,mie,19


In [4]:
table = (
    cleaned_data.groupby(["radicado", "clase_vehiculo"]).size().unstack(fill_value=0)
)

# Drop the column level 'clase_vehiculo'
table.columns.name = None

# Reset the index so 'radicado' becomes a column
table = table.reset_index()

table

,radicado,automovil,bicicleta,bus,camion,motocicleta,otros,retroexcavadora
0,1423926,0,0,1,0,0,0,0
1,1423928,0,0,1,0,0,0,0
2,1423988,0,0,1,0,0,0,0
3,1424000,1,0,1,0,0,0,0
4,1424009,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
45555,1739876,1,0,1,0,0,0,0
45556,1739890,0,0,1,0,0,0,0
45557,1739934,0,0,1,0,0,1,0
45558,1739935,1,0,1,0,0,0,0


In [5]:
# table = table.applymap(lambda x: 1 if x != 0 else x)


table["categories"] = table.apply(get_categories_with_number, axis=1)
table

,radicado,automovil,bicicleta,bus,camion,motocicleta,otros,retroexcavadora,categories
0,1423926,0,0,1,0,0,0,0,bus
1,1423928,0,0,1,0,0,0,0,bus
2,1423988,0,0,1,0,0,0,0,bus
3,1424000,1,0,1,0,0,0,0,automovil-bus
4,1424009,1,0,1,0,0,0,0,automovil-bus
...,...,...,...,...,...,...,...,...,...
45555,1739876,1,0,1,0,0,0,0,automovil-bus
45556,1739890,0,0,1,0,0,0,0,bus
45557,1739934,0,0,1,0,0,1,0,bus-otros
45558,1739935,1,0,1,0,0,0,0,automovil-bus


In [6]:
# Create a new column "SOLO BUS" based on the conditions
table["solo bus"] = table.apply(
    lambda row: 1
    if row["bus"] == 1
    and row["automovil"] == 0
    and row["bicicleta"] == 0
    and row["camion"] == 0
    and row["motocicleta"] == 0
    and row["otros"] == 0
    else 0,
    axis=1,
)
table

,radicado,automovil,bicicleta,bus,camion,motocicleta,otros,retroexcavadora,categories,solo bus
0,1423926,0,0,1,0,0,0,0,bus,1
1,1423928,0,0,1,0,0,0,0,bus,1
2,1423988,0,0,1,0,0,0,0,bus,1
3,1424000,1,0,1,0,0,0,0,automovil-bus,0
4,1424009,1,0,1,0,0,0,0,automovil-bus,0
...,...,...,...,...,...,...,...,...,...,...
45555,1739876,1,0,1,0,0,0,0,automovil-bus,0
45556,1739890,0,0,1,0,0,0,0,bus,1
45557,1739934,0,0,1,0,0,1,0,bus-otros,0
45558,1739935,1,0,1,0,0,0,0,automovil-bus,0


In [7]:
table.loc[table["solo bus"] == 1, "categories"] = "solo bus"
table

,radicado,automovil,bicicleta,bus,camion,motocicleta,otros,retroexcavadora,categories,solo bus
0,1423926,0,0,1,0,0,0,0,solo bus,1
1,1423928,0,0,1,0,0,0,0,solo bus,1
2,1423988,0,0,1,0,0,0,0,solo bus,1
3,1424000,1,0,1,0,0,0,0,automovil-bus,0
4,1424009,1,0,1,0,0,0,0,automovil-bus,0
...,...,...,...,...,...,...,...,...,...,...
45555,1739876,1,0,1,0,0,0,0,automovil-bus,0
45556,1739890,0,0,1,0,0,0,0,solo bus,1
45557,1739934,0,0,1,0,0,1,0,bus-otros,0
45558,1739935,1,0,1,0,0,0,0,automovil-bus,0


In [8]:
print(table["categories"].unique())

['solo bus' 'automovil-bus' 'bus-motocicleta' 'bus-camion' 'bus'
 'automovil-bus-motocicleta' 'automovil-bus-camion' 'bus-otros'
 'bus-camion-motocicleta' 'bicicleta-bus' 'automovil-bus-otros'
 'automovil-bicicleta-bus' 'bicicleta-bus-motocicleta'
 'automovil-bus-motocicleta-otros' 'bus-motocicleta-otros'
 'bus-camion-otros' 'bicicleta-bus-camion'
 'automovil-bus-camion-motocicleta' 'automovil-bicicleta-bus-motocicleta'
 'automovil-bus-camion-otros']


In [9]:
table_total = table.sum(axis=0)
table_total

radicado                                                 71829014424
automovil                                                      27690
bicicleta                                                        495
bus                                                            48868
camion                                                          1591
motocicleta                                                     8525
otros                                                            475
retroexcavadora                                                    1
categories         solo bussolo bussolo busautomovil-busautomovil...
solo bus                                                        6098
dtype: object

In [10]:
# Calculate the value counts for each category
conteo_categorias_tipo_vehiculo = table["categories"].value_counts()

# Calculate the total count of all categories
total_count = conteo_categorias_tipo_vehiculo.sum()

# Calculate the percentages for each category
conteo_categorias_tipo_vehiculo_percentages = [
    (category, round((count / total_count * 100), 2))
    for category, count in conteo_categorias_tipo_vehiculo.items()
]

conteo_categorias_tipo_vehiculo_percentages

[('automovil-bus', 56.72),
 ('bus-motocicleta', 16.22),
 ('solo bus', 13.38),
 ('bus', 6.41),
 ('bus-camion', 3.11),
 ('automovil-bus-motocicleta', 1.69),
 ('bicicleta-bus', 1.03),
 ('bus-otros', 0.89),
 ('automovil-bus-camion', 0.2),
 ('bus-camion-motocicleta', 0.13),
 ('automovil-bus-otros', 0.08),
 ('bus-motocicleta-otros', 0.04),
 ('automovil-bicicleta-bus', 0.03),
 ('bicicleta-bus-motocicleta', 0.02),
 ('automovil-bus-camion-motocicleta', 0.02),
 ('bus-camion-otros', 0.01),
 ('automovil-bus-motocicleta-otros', 0.01),
 ('bicicleta-bus-camion', 0.0),
 ('automovil-bicicleta-bus-motocicleta', 0.0),
 ('automovil-bus-camion-otros', 0.0)]